<a href="https://colab.research.google.com/github/caio-git/stance_detection/blob/main/Dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
import re
from urllib.parse import *
import pandas as pd
from zipfile import ZipFile
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
!wget -O /content/Fake.br-Corpus-master.zip https://github.com/caio-git/datasets/blob/main/Fake.br-Corpus-master.zip?raw=true

file_name = '/content/Fake.br-Corpus-master.zip'

with ZipFile(file_name, 'r') as z:    
    z.extractall()    

--2021-07-02 17:12:04--  https://github.com/caio-git/datasets/blob/main/Fake.br-Corpus-master.zip?raw=true
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/caio-git/datasets/raw/main/Fake.br-Corpus-master.zip [following]
--2021-07-02 17:12:05--  https://github.com/caio-git/datasets/raw/main/Fake.br-Corpus-master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/caio-git/datasets/main/Fake.br-Corpus-master.zip [following]
--2021-07-02 17:12:05--  https://raw.githubusercontent.com/caio-git/datasets/main/Fake.br-Corpus-master.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connecte

In [3]:
information = {0 : 'author',
1 : 'link',
2 : 'category',
3 : 'date of publication',
4 : 'number of tokens',
5 : 'number of words without punctuation',
6 : 'number of types',
7 : 'number of links inside the news',
8 : 'number of words in upper case',
9 : 'number of verbs',
10 : 'number of subjuntive and imperative verbs',
11 : 'number of nouns',
12 : 'number of adjectives',
13 : 'number of adverbs',
14 : 'number of modal verbs (mainly auxiliary verbs)',
15 : 'number of singular first and second personal pronouns',
16 : 'number of plural first personal pronouns',
17 : 'number of pronouns',
18 : 'pausality',
19 : 'number of characters',
20 : 'average sentence length',
21 : 'average word length',
22 : 'percentage of news with speeling errors',
23 : 'emotiveness',
24 : 'diversity'}

In [4]:
pages = []
for i in range(3602):
  if i not in (696,1467):
    pages.append(f'/content/full_texts/fake-meta-information/{i+1}-meta.txt')

li = []

for filename in pages:
    df = pd.read_csv(filename, encoding='utf8', index_col=None, header=None)
    df = df.transpose()
    li.append(df)

df_fake_meta_information = pd.concat(li, axis=0, ignore_index=True)
df_fake_meta_information.rename(columns=information, inplace=True)
#df_fake_meta_information.head()

In [5]:
pages = []
for i in range(3602):
  if i not in (696,1467):
    pages.append(f'/content/full_texts/true-meta-information/{i+1}-meta.txt')

le = []

for filename in pages:
    df = pd.read_csv(filename, encoding='utf8', index_col=None, header=None, delimiter='\t')
    df = df.transpose()
    df = df.dropna()
    le.append(df)

df_true_meta_information = pd.concat(le, axis=0, ignore_index=True)
df_true_meta_information.rename(columns=information, inplace=True)
#df_true_meta_information.head()

In [6]:
texts = []
for i in range(3602):
  if i not in (696,1467):
    f = open(f'/content/full_texts/fake/{i+1}.txt', 'r')
    texts.append(f.read())
df_fake_meta_information['news'] = texts
df_fake_meta_information['label'] = 'fake'
#df_fake_meta_information.head()

In [7]:
texts = []
for i in range(3602):
  if i not in (696,1467):
    f = open(f'/content/full_texts/true/{i+1}.txt', 'r')
    texts.append(f.read())
df_true_meta_information['news'] = texts
df_true_meta_information['label'] = 'true'
#df_true_meta_information.head()

In [8]:
df_fake_meta_information.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 27 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   author                                                 3600 non-null   object
 1   link                                                   3600 non-null   object
 2   category                                               3600 non-null   object
 3   date of publication                                    3600 non-null   object
 4   number of tokens                                       3600 non-null   object
 5   number of words without punctuation                    3600 non-null   object
 6   number of types                                        3600 non-null   object
 7   number of links inside the news                        3600 non-null   object
 8   number of words in upper case                          360

In [9]:
#df_fake_meta_information.head()

In [10]:
def extract_headline_fake(t):
  headline_fake = nltk.sent_tokenize(t)
  t = re.sub(r'\s+', ' ', headline_fake[0])
  t = re.sub(r'“|”', '', t)
  t = re.sub(r'\\x\d{2}|||||', '', t)
  texto_formatado = t.lower()
  return texto_formatado

In [11]:
df_fake_meta_information['headline_fake'] = df_fake_meta_information.news.apply(extract_headline_fake)

In [12]:
def extract_news_fake(t):
  news_fake = nltk.sent_tokenize(t)
  news_fake = ' '.join(news_fake[1:])
  t = re.sub(r'\s+', ' ', news_fake)
  t = re.sub(r'“|”', '', t)
  t = re.sub(r'\\x\d{2}|||||', '', t)
  texto_formatado = t.lower()
  return texto_formatado

In [13]:
df_fake_meta_information['news'] = df_fake_meta_information.news.apply(extract_news_fake)
#df_fake_meta_information.head()

In [14]:
df_fake = df_fake_meta_information[['news', 'label', 'headline_fake']]
df_fake = df_fake.rename(columns={'news':'news_fake', 'label':'label_fake'})
#df_fake.head()

In [15]:
df_true_meta_information.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 27 columns):
 #   Column                                                 Non-Null Count  Dtype 
---  ------                                                 --------------  ----- 
 0   author                                                 3600 non-null   object
 1   link                                                   3600 non-null   object
 2   category                                               3600 non-null   object
 3   date of publication                                    3600 non-null   object
 4   number of tokens                                       3600 non-null   object
 5   number of words without punctuation                    3600 non-null   object
 6   number of types                                        3600 non-null   object
 7   number of links inside the news                        3600 non-null   object
 8   number of words in upper case                          360

In [16]:
df_true_meta_information.head()

,author,link,category,date of publication,number of tokens,number of words without punctuation,number of types,number of links inside the news,number of words in upper case,number of verbs,number of subjuntive and imperative verbs,number of nouns,number of adjectives,number of adverbs,number of modal verbs (mainly auxiliary verbs),number of singular first and second personal pronouns,number of plural first personal pronouns,number of pronouns,pausality,number of characters,average sentence length,average word length,percentage of news with speeling errors,emotiveness,diversity,news,label
0,Naira Trindade,http://politica.estadao.com.br/blogs/coluna-do...,politica,13/12/2017,168,148,107,None,0,24,2,43,5,4,3,0,0,7,3.33333,761,24.6667,5.14189,0.0,0.134328,0.722973,﻿O Podemos decidiu expulsar o deputado federa...,true
1,Estadão Conteúdo,http://esportes.estadao.com.br/noticias/futebo...,sociedade_cotidiano,26/12/2017,349,294,182,None,3,60,1,72,19,24,4,0,0,29,2.75,1477,14.7,5.02381,0.0,0.325758,0.619048,Em evento realizado nesta terça-feira para div...,true
2,Juliana Diógenes,"http://politica.estadao.com.br/noticias/geral,...",politica,26/12/2017,249,204,128,None,0,38,1,62,10,6,4,3,0,8,3.0,1019,13.6,4.9951,0.0147059,0.16,0.627451,﻿Após o prefeito de Manaus Arthur Virgílio (PS...,true
3,Mônica Bergamo,http://www1.folha.uol.com.br/colunas/monicaber...,politica,11/1/2018,170,147,102,None,2,22,1,50,1,4,3,0,0,7,2.875,700,18.375,4.7619,0.0,0.0694444,0.693878,﻿Doria vai receber Zé Celso após reunião com r...,true
4,Luiz Vassallo E Breno Pires,http://politica.estadao.com.br/blogs/fausto-ma...,politica,28/12/2017,389,341,181,None,12,53,2,101,10,7,10,0,0,14,2.66667,1587,18.9444,4.65396,0.00879765,0.11039,0.530792,"Gustavo Pedreira Ferraz, que admitiu buscar ma...",true


In [17]:
def extract_headline_true(url):
  headline = urlparse(url)
  #print(headline.path)
  headline = re.sub(r'/$', '', headline.path)
  headline = re.sub(r'\.\w+', '', headline)
  headline = re.sub(r',\d+', '', headline)  
  headline = headline.split('/')
  headline = headline[-1]
  headline = re.sub(r'^\d+', '', headline)
  headline = headline.split(',')
  headline = headline[-1]
  headline = re.sub(r'\W', ' ', headline)
  #print(headline)
  #print()
  return headline

In [18]:
df_true_meta_information['headline_true'] = df_true_meta_information.link.apply(extract_headline_true)

In [19]:
df_true = df_true_meta_information[['news', 'label', 'headline_true']]
df_true = df_true.rename(columns={'news':'news_true', 'label':'label_true'})
#df_true.head()

In [45]:
df1 = pd.merge(df_fake, df_true, how='inner', on = df_fake.index)
#df1.head()

In [46]:
df1_disagree = df1.drop(columns=['label_fake', 'headline_fake', 'news_true', 'label_true', 'key_0'])
df1_disagree['tag'] = 'disagree'
#df1_disagree.head()

In [47]:
df2 = pd.merge(df_fake, df_true, how='inner', on = df_fake.index)
#df2.head()

In [48]:
df2_disagree = df2.drop(columns=['news_fake', 'label_fake', 'label_true', 'headline_true', 'key_0'])
df2_disagree['tag'] = 'disagree'
#df2_disagree.head()

In [49]:
df3_agree = pd.concat([df_fake.rename(columns={'news_fake':'news', 'headline_fake':'headline', 'label_fake':'label'}), 
                       df_true.rename(columns={'news_true':'news', 'headline_true':'headline', 'label_true':'label'})])
df3_agree['tag'] = 'agree'
df3_agree = df3_agree.drop(columns='label')
#df3_agree.head()

In [50]:
#df3_agree

In [51]:
df1_disagree.columns

Index(['news_fake', 'headline_true', 'tag'], dtype='object')

In [52]:
df1_disagree.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3599
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_fake      3600 non-null   object
 1   headline_true  3600 non-null   object
 2   tag            3600 non-null   object
dtypes: object(3)
memory usage: 112.5+ KB


In [53]:
df2_disagree.columns

Index(['headline_fake', 'news_true', 'tag'], dtype='object')

In [54]:
df2_disagree.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3600 entries, 0 to 3599
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   headline_fake  3600 non-null   object
 1   news_true      3600 non-null   object
 2   tag            3600 non-null   object
dtypes: object(3)
memory usage: 112.5+ KB


In [55]:
df3_agree.columns

Index(['news', 'headline', 'tag'], dtype='object')

In [56]:
df3_agree.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7200 entries, 0 to 3599
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   news      7200 non-null   object
 1   headline  7200 non-null   object
 2   tag       7200 non-null   object
dtypes: object(3)
memory usage: 225.0+ KB


In [57]:
df3_agree.tag.value_counts()

agree    7200
Name: tag, dtype: int64

In [58]:
df1_disagree_part = df1_disagree.iloc[0:1200,:].copy()
df1_disagree_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_fake      1200 non-null   object
 1   headline_true  1200 non-null   object
 2   tag            1200 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB


In [59]:
df2_disagree_part = df2_disagree.iloc[1200:2400,:].copy()
df2_disagree_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 1200 to 2399
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   headline_fake  1200 non-null   object
 1   news_true      1200 non-null   object
 2   tag            1200 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB


In [60]:
df3_agree_part = df3_agree.iloc[2400:3600,:].copy()
df3_agree_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 2400 to 3599
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   news      1200 non-null   object
 1   headline  1200 non-null   object
 2   tag       1200 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB


In [61]:
df4_agree_part = df3_agree.iloc[6000:,:].copy()
df4_agree_part.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 2400 to 3599
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   news      1200 non-null   object
 1   headline  1200 non-null   object
 2   tag       1200 non-null   object
dtypes: object(3)
memory usage: 37.5+ KB


In [62]:
df1_disagree_part = df1_disagree_part.rename(columns={'news_fake':'news', 'headline_true':'headline'})
df2_disagree_part = df2_disagree_part[['news_true', 'headline_fake', 'tag']]
df2_disagree_part = df2_disagree_part.rename(columns={'news_true':'news', 'headline_fake':'headline'})
df = pd.concat([df1_disagree_part, df2_disagree_part, df3_agree_part, df4_agree_part])
df

,news,headline,tag
0,a senadora kátia abreu (sem partido-to) disse ...,podemos expulsa gaguim e abre espaco para kati...,disagree
1,"um texto de jorge gauthier, para o blog me sal...",zico critica montagem do elenco no flamengo e ...,disagree
2,"o governador de são paulo, geraldo alckmin, as...",alckmin diz que previa nao divide previa escolhe,disagree
3,o teatro oficina procurou o ministério público...,doria vai receber ze celso apos reuniao com r...,disagree
4,a ministra dos direitos humanos luislinda valo...,laranja de geddel trabalhou na campanha de aecio,disagree
...,...,...,...
3595,Como a Bahia virou uma potência mundial do mar...,como a bahia virou uma potencia mundial do mar...,agree
3596,"Alvo da Lava Jato, Bendine tinha passagem comp...",alvo da lava jato bendine tinha passagem compr...,agree
3597,Chefs convidados do Encontro Mundial das Cidad...,chefs convidados do encontro mundial das cidad...,agree
3598,"As lições de educação moral e cívica, que o b...",joesley janot fachin temer cia,agree


In [63]:
df.tag.value_counts()

agree       2400
disagree    2400
Name: tag, dtype: int64

In [64]:
df_drop_indexes = df[df['news'].apply(len) < 50].index
df_drop_indexes

Int64Index([ 402,  479,  585,  649,  786,  976, 1093, 1140, 2424, 2444, 2496,
            2757, 3225, 3297, 3307, 3383, 3440, 3492, 3515, 3518, 3536, 3547],
           dtype='int64')

In [65]:
df.drop(df_drop_indexes, inplace=True)
df

,news,headline,tag
0,a senadora kátia abreu (sem partido-to) disse ...,podemos expulsa gaguim e abre espaco para kati...,disagree
1,"um texto de jorge gauthier, para o blog me sal...",zico critica montagem do elenco no flamengo e ...,disagree
2,"o governador de são paulo, geraldo alckmin, as...",alckmin diz que previa nao divide previa escolhe,disagree
3,o teatro oficina procurou o ministério público...,doria vai receber ze celso apos reuniao com r...,disagree
4,a ministra dos direitos humanos luislinda valo...,laranja de geddel trabalhou na campanha de aecio,disagree
...,...,...,...
3595,Como a Bahia virou uma potência mundial do mar...,como a bahia virou uma potencia mundial do mar...,agree
3596,"Alvo da Lava Jato, Bendine tinha passagem comp...",alvo da lava jato bendine tinha passagem compr...,agree
3597,Chefs convidados do Encontro Mundial das Cidad...,chefs convidados do encontro mundial das cidad...,agree
3598,"As lições de educação moral e cívica, que o b...",joesley janot fachin temer cia,agree


In [66]:
df.sample(10)

,news,headline,tag
566,a força aérea dos estados unidos ordenou que s...,qual a ameaca real do torpedo do juizo final a...,disagree
2845,Em alegações finais enviadas à Justiça nesta ...,procuradoria diz que delcidio mentiu e quer co...,agree
3504,Complexo Esportivo Canhoteiro em São Luís sofr...,complexo esportivo do canhoto em sao luis sofr...,agree
2430,CPF em troca de desconto: MP investiga venda d...,cpf em troca de desconto mp investiga venda de...,agree
3264,"a jornalista claudia cruz, esposa do president...",esposa de cunha tentou contato com grandes emi...,agree
1114,quem derrubará temer? cunha ou funaro?. preso ...,politica,disagree
2688,processos estão a um passo de curitiba. teori ...,saiba o que anda tirando o sono de lula.,agree
3077,a presidente dilma pode estar trabalhando na m...,jornalista diz que dilma pode estar trabalhand...,agree
1335,"Com denúncia rejeitada, governo quer aprovar r...",policiais invadem o congresso nacional em prot...,disagree
1627,OMS diz que nenhum caso de zika ocorreu na Oli...,"rússia faz alerta sobre zika virus: ""porcaria ...",disagree


In [67]:
df.to_csv('/content/stance_detection_3.csv', index=False)